In [1]:
from typing_extensions import TypedDict
from typing import Annotated, List
from operator import add
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from IPython.display import Image
import math

class State(TypedDict):
    num: int
    arr: Annotated[List[int], add]    
 
def continue_dynamic(state: State): 
    return [Send("dynamic_node", {"current_num": i}) for i in range(1,state["num"]+1)]

def dynamicNode(state: dict):
    current_num = state["current_num"]
    factorial = math.factorial(current_num)
    print(f'Calculating factorial of {current_num}: {factorial}')
    return {"arr": [factorial]}

graph = StateGraph(State) 
graph.add_node("dynamic_node", dynamicNode) 
graph.add_conditional_edges(START, continue_dynamic, ["dynamic_node"])
graph.add_edge("dynamic_node", END)

compiled_graph = graph.compile()
mermaid_code = compiled_graph.get_graph().draw_mermaid_png()
display(Image(mermaid_code)) 
result = compiled_graph.invoke({"num": 10})
print("Final result:", result)


ModuleNotFoundError: No module named 'langgraph'

In [ ]:
from typing_extensions import TypedDict
from typing import Annotated, List
import operator
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from langgraph.errors import InvalidUpdateError
from IPython.display import Image, display  
class State(TypedDict):
    valuelist: Annotated[List[str], operator.add] 

class SubState(TypedDict):
    valuelist :  str
def node_function(state: State):
   pass
 
def sub_node_function(state: dict):  
    current_value = state['valuelist']   
    print(f'Processing sub-node with value: {current_value}') 
    return {'valuelist': [current_value]}  
def create_parallel_routes(state: State):
    parallel_nodes = ['b','c'] 
    return [Send('sub_builder', {'valuelist': i}) for i in parallel_nodes]
 
sub_builder = StateGraph(SubState)
sub_builder.add_node('sub_node', sub_node_function)
sub_builder.add_edge(START, 'sub_node')
sub_builder.add_edge('sub_node', END)
 
builder = StateGraph(State)
builder.add_node('a', node_function)
builder.add_node('sub_builder', sub_builder.compile())
builder.add_node('e', node_function)
builder.add_edge(START, 'a')
builder.add_conditional_edges('a', create_parallel_routes, ['sub_builder'])
builder.add_edge('sub_builder', 'e')
builder.add_edge('e', END)

graph = builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

try: 
    resp = graph.invoke({"valuelist": ['a']})
    print("Final result:", resp)
except InvalidUpdateError as e:
    print(f"An error occurred: {e}")


In [ ]:
from typing_extensions import TypedDict
from typing import Annotated, List
from operator import add
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from IPython.display import Image
import math

class State(TypedDict):
    nums: list[int|None]
    sum:int
    final_fact: Annotated[List[int], add]   
    final_double: Annotated[List[int], add]
    final_tripple: Annotated[List[int], add]
class SubState(TypedDict):
    final_double:int
    final_tripple:int
    final_fact: int
    current_num:int

def is_prime(n):
     if n <= 1:
         return False
     for i in range(2, int(math.sqrt(n)) + 1):
         if n % i == 0:
             return False
     return True 

def prime_filter(state: State):
    final_list = []
    for i in state["nums"]:
        if is_prime(i):
            final_list.append(i)
    print(f'final_list:{final_list}')        
    return {"nums": final_list}
     
def continue_dynamic(state: State): 
    finalLIst = []
    for i in state["nums"]:
      finalLIst.append(Send("dynamic_node", {"current_num": i}))
    return finalLIst

def findfactorial(state: dict):
    current_num = state["current_num"]
    factorial = math.factorial(current_num) 
    return {"final_fact": [factorial] }

def doubler(state:SubState): 
  final_res = state["current_num"] * 2
  print(f'doubler:{final_res}')
  return {"final_double":[final_res], }

def tripple(state:SubState): 
  final_res = state["current_num"] * 3
  print(f'tripple:{final_res} type {type(state["current_num"])}')
  return {"final_tripple":[final_res], }

def finalSum(state:State):
    print(f'finalSum:{state["final_fact"]}') 
    return {"sum":sum(state["final_fact"])}


subGraph = StateGraph(SubState)
subGraph.add_node("findfactorial", findfactorial)
subGraph.add_node("doubler",doubler)
subGraph.add_node("tripple",tripple)
subGraph.add_edge(START, "findfactorial")
subGraph.add_edge("findfactorial", 'doubler')
subGraph.add_edge("doubler", 'tripple')
subGraph.add_edge("tripple", END)


graph = StateGraph(State) 
graph.add_node("prime_filter", prime_filter) 
graph.add_node("dynamic_node", subGraph.compile()) 
graph.add_node("final_sum",finalSum)

graph.add_edge(START, "prime_filter")
graph.add_conditional_edges("prime_filter", continue_dynamic, ["dynamic_node"])
graph.add_edge("dynamic_node", "final_sum")
graph.add_edge("final_sum", END)

compiled_graph = graph.compile()
mermaid_code = compiled_graph.get_graph(xray=1).draw_mermaid_png()
display(Image(mermaid_code)) 
result = compiled_graph.invoke({"nums": [3,6,7,8,9]})
print("Final result:", result)


In [ ]:
from typing_extensions import TypedDict
from typing import Annotated, List
import operator
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from langgraph.errors import InvalidUpdateError
from IPython.display import Image, display

class State(TypedDict):
    inputList:List[str]
    valuelist: Annotated[List[str], operator.add]
    finalList: Annotated[List[str], operator.add]
    finalStr:str

class SubState(TypedDict):
    current_value: str
    finalList: str

def node_function(state: State):
    new_list = [word.upper() for word in state['inputList']] 
    return {'valuelist': new_list} 

def main_node_function(state: State):
    current_list = state['valuelist']
    return [Send("subgraph", {"current_value": i}) for i in current_list]

def sub_node_1(state: SubState):
    current_value = state['current_value']
    new_value = current_value+current_value
    return {'current_value': new_value}

def sub_node_2(state: SubState):
    current_value = state['current_value']
    new_value = current_value+current_value
    return {'finalList': [new_value]}
 

def merger_node(state: State):
     return {'finalStr':"".join(state['finalList'])}
 
subgraph = StateGraph(SubState)
subgraph.add_node("sub_node_1", sub_node_1)
subgraph.add_node("sub_node_2", sub_node_2)
subgraph.add_edge(START, "sub_node_1")
subgraph.add_edge("sub_node_1", "sub_node_2")
subgraph.add_edge("sub_node_2", END)

main_graph = StateGraph(State)
main_graph.add_node("main_node", node_function)
main_graph.add_node("subgraph", subgraph.compile())
main_graph.add_node("merger_node", merger_node)
main_graph.add_edge(START, "main_node")
main_graph.add_conditional_edges("main_node",main_node_function, ["subgraph"])
main_graph.add_edge("subgraph", 'merger_node')
main_graph.add_edge('merger_node',END)

graph = main_graph.compile()
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

resp = graph.invoke({"inputList": ['a','b','c']})
print("Final result:", resp)